## Librerias

In [8]:
import pyodbc
import pandas as pd
import numpy as np
import os
import glob
import warnings
from tabulate import tabulate


# Para quitar unos warnings
warnings.filterwarnings("ignore", message="pandas only supports SQLAlchemy connectable")


## Pruebas de funcionamiento

**Importacion de las Consultas SQL**

Prueba para ver si se importan de forma correcta los arhivos que tenemos en la carpeta queries/.Solo tomamos aquellos que vamos a utilizar para nuestro modelo dimensional,es decir los que empiezan por dim_

In [9]:
BASE_DIR = os.getcwd()  
queries_path = os.path.join(BASE_DIR, "..", "queries")


dim_files = glob.glob(os.path.join(queries_path, "dim_*.sql"))

if not dim_files:
    print("ERROR: No se encontraron archivos que empiecen con 'dim_'.")
else:
    print(f"Encontrados {len(dim_files)} archivos:")
    for sql_file in dim_files:
        with open(sql_file, "r", encoding="utf-8") as f:
            sql_content = f.read()
        print(f"Archivo: {os.path.basename(sql_file)}")


Encontrados 5 archivos:
Archivo: dim_cliente.sql
Archivo: dim_fact.sql
Archivo: dim_geog.sql
Archivo: dim_prod.sql
Archivo: dim_tiempo.sql


## Exportación Tablas Azure-Local

**Conexiones y Rutas de las Consultas**

Este código establece las conexiones necesarias para extraer las tablas de Azure SQL Database y utilizar esas tablas para ejecutar las consultas necesarias que crean las tablas de nuestro modelo dimensional. Estas se cargarán en una base de datos local (SSMS).

- Servidores:
  - Azure SQL Database: Servidor en la nube (tablas originales)
  - SQL Server Local (SSMS): Servidor local donde se crean las tablas del modelo dimensional

- Driver:
  - Se utiliza el ODBC Driver 17 for SQL Server, que viene por defecto en Windows y permite conectar y ejecutar consultas en ambas instancias.


In [10]:
#Conexión Azure
AZURE_SERVER = 'uaxmathfis.database.windows.net'
AZURE_DATABASE = 'usecases'
AZURE_DRIVER = '{ODBC Driver 17 for SQL Server}'
azure_conn_str = f"DRIVER={AZURE_DRIVER};SERVER={AZURE_SERVER};DATABASE={AZURE_DATABASE};Authentication=ActiveDirectoryInteractive"

# Consexión SQL Server Local
LOCAL_SERVER = 'localhost'
LOCAL_DATABASE = 'dwh_case1'
LOCAL_DRIVER = '{ODBC Driver 17 for SQL Server}'
local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# Rutas a las consutlas para modelo dimensional
BASE_DIR = os.getcwd()
queries_path = os.path.join(BASE_DIR, "..", "queries")
dim_files = glob.glob(os.path.join(queries_path, "dim_*.sql"))

Establecemos las conexiones con Azure y SSMS

In [11]:
conn_azure = pyodbc.connect(azure_conn_str)
conn_local = pyodbc.connect(local_conn_str)
cursor_local = conn_local.cursor()
print("Conexiones establecidas")

Conexiones establecidas


**ETL**

Este código realiza un proceso ETL completo para migrar datos desde Azure a SQL Server Local:

- *Extract:*  
  Se conecta a Azure SQL y, para cada archivo SQL en la carpeta "queries", se ejecuta la consulta para extraer los datos (en forma de DataFrame).

- *Transform*  
  Se limpian los datos (se reemplazan nulos por 0) y se ajustan los tipos de datos (por ejemplo, de float64 a float32 y de int64 a int32) para optimizar la inserción en SQL Server.

- *Load*  
  Se elimina la tabla local (si existe), se crea una nueva tabla en SQL Server Local usando una definición dinámica basada en el DataFrame, y se insertan los datos extraídos.



In [12]:
resumen_tablas = []

for sql_file in dim_files:
    table_name = os.path.splitext(os.path.basename(sql_file))[0]
    NEW_TABLE_NAME = table_name  
    
    # EXTRACT: Leemos  la consulta y extraemos datos desde Azure SQL
    with open(sql_file, 'r', encoding='utf-8') as file:
        sql_query = file.read()
    df = pd.read_sql(sql_query, conn_azure)
    
    if df.empty:
        print(f"Tabla: {NEW_TABLE_NAME} sin datos, se omite.")
        continue
    
    filas_extracted, columnas_extracted = df.shape

    # TRANSFORM: Tratamiento tipos datos

    # Tratamiento nulos

    # 1. Reemplazar celdas vacías o con espacios por NaN
    df = df.replace(r'^\s*$', np.nan, regex=True)

    # 2. Tratamiento personalizado para columnas categóricas y sensibles
    if 'GENERO' in df.columns:
        df['GENERO'] = df['GENERO'].fillna('NA')  # Evita 0 para análisis categórico en Power BI

    if 'provincia' in df.columns:
        df['provincia'] = df['provincia'].fillna('NA')  # Mejor para filtros y agrupaciones

    if 'QUEJA' in df.columns:
        df['QUEJA'] = df['QUEJA'].fillna('NA')  # Así se distingue de 'NO'

    if 'lat' in df.columns:
        df['lat'] = df['lat'].fillna('NA')  # Evita errores en mapas (0,0)

    if 'lon' in df.columns:
        df['lon'] = df['lon'].fillna('NA')

    if 'Fecha_Ultima_Revision' in df.columns:
        df['Fecha_Ultima_Revision'] = pd.to_datetime(df['Fecha_Ultima_Revision'], errors='coerce')  # Deja como NaT los nulos

    # 3. Aplicar fillna(0) al resto del DataFrame
    columnas_excluidas = ['lat', 'lon', 'Fecha_Ultima_Revision', 'GENERO', 'provincia', 'QUEJA']
    columnas_a_rellenar = [col for col in df.columns if col not in columnas_excluidas]

    df[columnas_a_rellenar] = df[columnas_a_rellenar].fillna(0)

    # 4. Asegurarnos de que todas las columnas con formato de fecha sean datetime
    columnas_fecha = ['Fecha_Ultima_Revision', 'Sales_Date', 'FIN_GARANTIA', 'BASE_DATE', 'Prod_date', 'Logistic_date']  # Añade más si es necesario

    for col in columnas_fecha:
        if col in df.columns:
            df[col] = pd.to_datetime(df[col], errors='coerce')

    #5. Tipos de datos numéricos
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = df[col].astype(np.float32)
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = df[col].astype(np.int32)

    # LOAD: Carga de las tablas del modelo dimensional en BBDD local
    cursor_local.execute(f"DROP TABLE IF EXISTS {NEW_TABLE_NAME}")  # Eliminamos tabla (si ya existe)
    conn_local.commit()
    print(f"-Tabla {NEW_TABLE_NAME} creada correctamente en SQL Server Local.")

    # Creamos la tabla con los tipos de datos ajustados
    create_table_sql = f"""
    CREATE TABLE {NEW_TABLE_NAME} (
        {', '.join([
            f'[{col}] DATE' if np.issubdtype(df[col].dtype, np.datetime64)
            else f'[{col}] FLOAT' if df[col].dtype == np.float32 
            else f'[{col}] INT' if df[col].dtype == np.int32 
            else f'[{col}] NVARCHAR(255)' for col in df.columns
        ])}
    );
    """
    cursor_local.execute(create_table_sql)
    conn_local.commit()
    
    # Insertamos datos en la tabla
    placeholders = ', '.join(['?' for _ in df.columns])  # Rellenamos con interrogaciones para luego insertar los datos
    insert_sql = f"INSERT INTO {NEW_TABLE_NAME} VALUES ({placeholders})"

    cursor_local.fast_executemany = True
    cursor_local.executemany(insert_sql, df.values.tolist())  # Creamos una lista con los valores del df y los introducimos en las tablas
    conn_local.commit()
    filas_insertadas = df.shape[0]
    
    # Mostramos un resumen de los datos extraídos para el modelo dimensional
    print(f" Tabla: {NEW_TABLE_NAME}")
    print(f"  Columnas extraídas: {columnas_extracted}")
    print(f"  Filas extraídas: {filas_extracted}")
    print(f"  Tabla eliminada (si existía): Sí")
    print(f"  Tabla creada: Sí")
    print(f"  Filas insertadas: {filas_insertadas}\n")
    
    resumen_tablas.append({
        "Tabla": NEW_TABLE_NAME,
        "Columnas extraídas": columnas_extracted,
        "Filas extraídas": filas_extracted,
        "Tabla eliminada": "Sí",
        "Tabla creada": "Sí",
        "Filas insertadas": filas_insertadas
    })

print("\n Modelo dimensional creado correctamente!")


-Tabla dim_cliente creada correctamente en SQL Server Local.
 Tabla: dim_cliente
  Columnas extraídas: 37
  Filas extraídas: 44053
  Tabla eliminada (si existía): Sí
  Tabla creada: Sí
  Filas insertadas: 44053

-Tabla dim_fact creada correctamente en SQL Server Local.
 Tabla: dim_fact
  Columnas extraídas: 41
  Filas extraídas: 58049
  Tabla eliminada (si existía): Sí
  Tabla creada: Sí
  Filas insertadas: 58049

-Tabla dim_geog creada correctamente en SQL Server Local.
 Tabla: dim_geog
  Columnas extraídas: 6
  Filas extraídas: 12
  Tabla eliminada (si existía): Sí
  Tabla creada: Sí
  Filas insertadas: 12

-Tabla dim_prod creada correctamente en SQL Server Local.
 Tabla: dim_prod
  Columnas extraídas: 17
  Filas extraídas: 404
  Tabla eliminada (si existía): Sí
  Tabla creada: Sí
  Filas insertadas: 404

-Tabla dim_tiempo creada correctamente en SQL Server Local.
 Tabla: dim_tiempo
  Columnas extraídas: 14
  Filas extraídas: 3652
  Tabla eliminada (si existía): Sí
  Tabla creada: Sí

**Resumen de las tablas importadas**

In [13]:
print(tabulate(resumen_tablas, headers="keys", tablefmt="grid", numalign="center"))
print("\nProceso completado correctamente!")


+-------------+----------------------+-------------------+-------------------+----------------+--------------------+
| Tabla       |  Columnas extraídas  |  Filas extraídas  | Tabla eliminada   | Tabla creada   |  Filas insertadas  |
+=============+======================+===================+===================+================+====================+
| dim_cliente |          37          |       44053       | Sí                | Sí             |       44053        |
+-------------+----------------------+-------------------+-------------------+----------------+--------------------+
| dim_fact    |          41          |       58049       | Sí                | Sí             |       58049        |
+-------------+----------------------+-------------------+-------------------+----------------+--------------------+
| dim_geog    |          6           |        12         | Sí                | Sí             |         12         |
+-------------+----------------------+-------------------+------

## Comprobaciones Finales

Es muy importante **mantener la consistencia** de los datos entre el origen (Azure ) y el destino (SSMS), ya que una pérdida de registros podría afectar directamente a los análisis posteriores.

Por eso, realizamos una verificación de integridad, comparando el número de filas que tiene cada tabla en Azure con las tablas generadas localmente tras ejecutar las consultas del modelo dimensional.


Realizaremos la compración  de esta forma:
| **Local Table (SSMS)** | **Azure Table**             |
|------------------------|-----------------------------|
| [dbo].[fact_table]   | [DATAEX].[001_sales]      |
| [dbo].[cliente]      | [DATAEX].[003_clientes]   |
| [dbo].[prod]         | [DATAEX].[006_producto]   |
| [dbo].[geog]        | [DATAEX].[011_tienda]     |

In [14]:
# Tablas a comparar
tables = {
    "[dbo].[dim_fact]": "[DATAEX].[001_sales]",
    "[dbo].[dim_cliente]":    "[DATAEX].[003_clientes]",
    "[dbo].[dim_prod]":       "[DATAEX].[006_producto]",
    "[dbo].[dim_geog]":       "[DATAEX].[011_tienda]"
}

# Lista para guardar resultados
resultados = []


for local_table, azure_table in tables.items():
        # Contar filas en Azure
        azure_query = f"SELECT COUNT(*) AS Total_Filas FROM {azure_table}"
        df_azure = pd.read_sql(azure_query, conn_azure)
        azure_count = df_azure['Total_Filas'].iloc[0]

        # Contar filas en Local
        local_query = f"SELECT COUNT(*) AS Total_Filas FROM {local_table}"
        df_local = pd.read_sql(local_query, conn_local)
        local_count = df_local['Total_Filas'].iloc[0]

        # Comparación
        resultado = "Coinciden" if azure_count == local_count else "No coinciden"
        nombre_tabla = local_table.split('.')[-1].replace('[', '').replace(']', '')
        resultados.append([nombre_tabla, azure_count, local_count, resultado])



# Mostrar tabla con resultados
print("\nResumen de comparación de tablas:\n")
print(tabulate(resultados, headers=["Tabla", "Azure SQL", "SQL Local", "Resultado"], tablefmt="grid"))


Resumen de comparación de tablas:

+-------------+-------------+-------------+-------------+
| Tabla       |   Azure SQL |   SQL Local | Resultado   |
+=============+=============+=============+=============+
| dim_fact    |       58049 |       58049 | Coinciden   |
+-------------+-------------+-------------+-------------+
| dim_cliente |       44053 |       44053 | Coinciden   |
+-------------+-------------+-------------+-------------+
| dim_prod    |         404 |         404 | Coinciden   |
+-------------+-------------+-------------+-------------+
| dim_geog    |          12 |          12 | Coinciden   |
+-------------+-------------+-------------+-------------+
